In [ ]:
import pandas as pd
import numpy as np
import re
import datetime as time
from collections import Counter
from mlxtend.frequent_patterns import apriori, association_rules
pd.set_option('display.max_columns', None)
df = pd.read_parquet('scontrini.parquet',engine='pyarrow')
df

In [ ]:
def mostCommonWord(s):
    words=s.replace('.',' ').split()
    returnList=[]
    for word in words:
        if (re.match(r"\b[A-Z]{3,}",word) is not None):
            returnList.append(word)
    return [x[0] for x in Counter(returnList).most_common(1)].pop()

def hot_encode(x):
    if(x<= 0):
        return 0
    if(x> 0):
        return 1

In [ ]:
for rep in df.reparto.unique():
        for sottRep in df[df.reparto==rep].sottoreparto.unique():
            df.loc[
                (df.reparto==rep)&
                (df.sottoreparto==sottRep)
                ,'descrizione']=mostCommonWord(
                df[(df.reparto==rep)&(df.sottoreparto==sottRep)].descrizione.to_string())

creazione tabella transazioni di novembre

In [ ]:
transazioniNov=df[['cassa','transazione','data_reg','descrizione','qta_riga']][(df.descrizione!="SERVIZIO")&(df.tipo=="")&(df.data_reg>=time.date(2021,11,1))&(df.data_reg<=time.date(2021,11,30))]

transazioniNov=transazioniNov.groupby(['cassa','transazione','data_reg','descrizione'])['qta_riga'].sum().unstack().reset_index().fillna(0).drop(['cassa', 'transazione','data_reg'], axis=1)
basketNov = transazioniNov.applymap(hot_encode)
basketNov

Creazione tabella transazioni dicembre

In [ ]:
transazioniDic=df[['cassa','transazione','data_reg','descrizione','qta_riga']][(df.descrizione!="SERVIZIO")&(df.tipo=="")&(df.data_reg>=time.date(2021,12,1))&(df.data_reg<=time.date(2021,12,31))]

transazioniDic=transazioniDic.groupby(['cassa','transazione','data_reg','descrizione'])['qta_riga'].sum().unstack().reset_index().fillna(0).drop(['cassa', 'transazione','data_reg'], axis=1)
basketDic = transazioniDic.applymap(hot_encode)
basketDic

Apriori applicato al basket di novembre

In [ ]:
frq_items_nov = apriori(basketNov, min_support = 0.01, use_colnames = True)
rules_nov = association_rules(frq_items_nov, metric ="lift", min_threshold = 1)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(rules_nov.sort_values(by=['confidence'], ascending=False))

Apriori applicato al basket di dicembre

In [ ]:
frq_items_dic = apriori(basketDic, min_support = 0.01, use_colnames = True)
rules_dic = association_rules(frq_items_dic, metric ="lift", min_threshold = 1)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(rules_dic.sort_values(by=['confidence'], ascending=False))